### Task 5

<span style="color: green; font-size: 14px;">

Итак... давай, наконец, выясним, повлияло ли посещение ленты новостей на поведение учащихся. 
Приступили ли они в итоге к работе над лабораторным заданием раньше? Помни, что у нас есть две подготовленные 
таблицы в базе данных: `test` и `control`. Мы выполним нечто, похожее на A/B-тестирование.
Чтобы обнаружить эффект, нам нужно вычислить значение дельты (период времени между датой первого коммита и
сроком сдачи лабораторного задания) до того момента, когда учащиеся впервые посетили страницу с лентой новостей, и после этого.
Мы должны сделать то же самое и для контрольной группы.

Другими словами, каждый пользователь из тестовой таблицы имеет свою собственную временную метку для
первого посещения новостной ленты. Мы хотим вычислить среднее значение дельты 
(разницу между датой первого коммита и сроком сдачи) до этой временной метки и после нее. 
Мы сделаем то же самое для пользователей в контрольной группе. Ты можешь сказать: «Но они вообще не посещали ленту новостей». 
Это так, и ранее мы решили использовать среднюю временную метку первого просмотра пользователями тестовой группы для пользователей контрольной группы.

Если значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого 
посещения в тестовой группе, и мы не видим аналогичного эффекта в контрольной группе, значит, создание страницы 
с новостной лентой было отличной идеей. Мы можем распространить эту практику на всю группу.


Подробное описание:
</span>
<span style="color: green; font-size: 13px;">

1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.
</span>

In [5]:
import sqlite3

# cоздаём соединение с базой данных
conn = sqlite3.connect('data/checking-logs.sqlite')

<span style="color: green; font-size: 13px;">

2. Используя только **один** запрос для каждой из групп, создай два датафрейма: `test_results` и `control_results` со столбцами `time` и `avg_diff` и только двумя строками.
    - `times` должно иметь значения `before` и `after`.
    - `avg_diff` содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
    - Учитываются только те пользователи, для которых имеются наблюдения и `before`, и `after`.
</span>

In [6]:
import pandas as pd

def read_query_with_table(table_name):
    """
    формирует SQL-запрос для извлечения данных из указанной таблицы
    
    Args:
        table_name (str): имя таблицы
    
    Returns:
        str: SQL-запрос
    """
    query = f"""
    SELECT time, AVG(avg_diff) AS avg_diff
    FROM (
        SELECT uid, labname,
            AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff,
            CASE
                WHEN (strftime('%s', first_commit_ts) < strftime('%s', first_view_ts)) THEN 'before'
                ELSE 'after'
            END AS time
        FROM {table_name}
        LEFT JOIN deadlines ON {table_name}.labname = deadlines.labs
        WHERE labname NOT LIKE 'project1'
        GROUP BY uid, time
    )
    WHERE uid IN (
        SELECT uid
        FROM (
            SELECT uid,
                CASE
                    WHEN (strftime('%s', first_commit_ts) < strftime('%s', first_view_ts)) THEN 'before'
                    ELSE 'after'
                END AS time
            FROM {table_name}
            LEFT JOIN deadlines ON {table_name}.labname = deadlines.labs
            WHERE labname NOT LIKE 'project1'
            GROUP BY uid, time
        )
        GROUP BY uid
        HAVING COUNT(uid) = 2
    )
    GROUP BY time;
    """
    return query

# запрос и извлечение данных для таблицы 'test'
test_results = pd.io.sql.read_sql(read_query_with_table('test'), conn)

# запрос и извлечение данных для таблицы 'control'
control_results = pd.io.sql.read_sql(read_query_with_table('control'), conn)

print(test_results, '\n')
print(control_results)

     time   avg_diff
0   after -99.523810
1  before -66.047619 

     time   avg_diff
0   after -99.322222
1  before -98.033333


<span style="color: green; font-size: 13px;">

3. Мы по-прежнему не используем лабораторное задание 'project1'.
4. Закрой соединение.
5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?
</span>

In [7]:
# закрываем соединение
conn.close()

наличие страницы с новостной лентой положительно влияет на поведение учащихся в группе "test", т.к. этой группе наблюдается заметная разница в среднем значении (avg_diff), приэтом в группе "control" разница несущественная